# 0. IMPORTS

## 0.1. Libraries

In [45]:
import pandas as pd
import numpy as np
import seaborn as sns

## 0.2. Datasets

### 0.2.1. Transactions

In [46]:
df_transactions = pd.read_csv('data/transactions.csv')

In [47]:
df_transactions.head()

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242


In [48]:
print('Number of rows: {}'.format(df_transactions.shape[0]))
print('Number of columns: {}'.format(df_transactions.shape[1]))

Number of rows: 337016
Number of columns: 4


### 0.2.2. Customers

In [49]:
df_customers = pd.read_csv('data/customers.csv')

In [50]:
df_customers.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [51]:
print('Number of rows: {}'.format(df_customers.shape[0]))
print('Number of columns: {}'.format(df_customers.shape[1]))

Number of rows: 8623
Number of columns: 3


### 0.2.3. Products

In [52]:
df_products = pd.read_csv('data/products.csv')

In [53]:
df_products.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [54]:
print('Number of rows: {}'.format(df_products.shape[0]))
print('Number of columns: {}'.format(df_products.shape[1]))

Number of rows: 3287
Number of columns: 3


## 0.3. Graph visualization

In [55]:
from IPython.core.display import HTML

def jupyter_settings():
   %matplotlib inline
   %pylab inline
   plt.style.use( 'bmh' )
   plt.rcParams['figure.figsize'] = [18, 8]
   plt.rcParams['font.size'] = 20
   display( HTML( '<style>.container { width:100% !important; }</style>') )
   pd.options.display.max_columns = None
   pd.options.display.max_rows = None
   pd.set_option( 'display.expand_frame_repr', False )
   sns.set()
    
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


# 1. DATA CLEANING

## 1.1. Creating main DataFrame

In [180]:
df1 = df_transactions.merge(df_customers, how = 'left', on = 'client_id')
df1 = df1.merge(df_products, how = 'left', on = 'id_prod')

#reordering columns
columns_ordered = ['id_prod', 'price', 'categ', 'date', 'client_id', 'sex', 'birth']
df1 = df1[columns_ordered]

In [181]:
df1.head()

,id_prod,price,categ,date,client_id,sex,birth
0,0_1483,4.99,0.0,2021-04-10 18:37:28.723910,c_4450,f,1977
1,2_226,65.75,2.0,2022-02-03 01:55:53.276402,c_277,f,2000
2,1_374,10.71,1.0,2021-09-23 15:13:46.938559,c_4270,f,1979
3,0_2186,4.20,0.0,2021-10-17 03:27:18.783634,c_4597,m,1963
4,0_1351,8.99,0.0,2021-07-17 20:34:25.800563,c_1242,f,1980


## 1.2. Data Dimensions

In [182]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of columns: {}'.format(df1.shape[1]))

Number of rows: 337016
Number of columns: 7


## 1.3. Check NAs

In [183]:
df1.isna().sum()

id_prod        0
price        103
categ        103
date           0
client_id      0
sex            0
birth          0
dtype: int64

In [184]:
# check if NA values for `price` and `categ` are in the same rows
df_na = df1[df1['price'].isna() | df1['categ'].isna()]
df_na.shape[0]

103

In [185]:
# check which products have NA values
df_na['id_prod'].unique()

array(['0_2245'], dtype=object)

In [186]:
df_na.head()

,id_prod,price,categ,date,client_id,sex,birth
6235,0_2245,NaN,NaN,2021-06-17 03:03:12.668129,c_1533,m,1972
10802,0_2245,NaN,NaN,2021-06-16 05:53:01.627491,c_7954,m,1973
14051,0_2245,NaN,NaN,2021-11-24 17:35:59.911427,c_5120,f,1975
17486,0_2245,NaN,NaN,2022-02-28 18:08:49.875709,c_4964,f,1982
21078,0_2245,NaN,NaN,2021-03-01 00:09:29.301897,c_580,m,1988


### 1.3.1. Fillout NAs
Empty features: `price` and `categ` for `id_prod` == '0_2245'

In [187]:
# does the dataframe possess a row with price and category for this product?
# answer: no, because the same number of rows is given for the NA dataframe and the product lines
df1[df1['id_prod'] == '0_2245'].shape[0]

103

In [188]:
print('Average year birth of clients: {}'.format(df1[df1['id_prod'] == '0_2245']['birth'].mean().round()))
print('Predominant sex of clients: {}'.format(df1[df1['id_prod'] == '0_2245']['sex'].mode()[0]))

Average year birth of clients: 1977.0
Predominant sex of clients: f


In [189]:
df_f_1977_avg = df1[(df1['birth'] == 1977) & (df1['sex'] == 'f')]
print('Average price of products sold for female clients born in 1977: $ {}'.format(df_f_1977_avg['price'].mean().round(2)))
print('Predominant category of products sold for female clients born in 1977: {}'.format(df_f_1977_avg['categ'].mode()[0]))

Average price of products sold for female clients born in 1977: $ 12.99
Predominant category of products sold for female clients born in 1977: 0.0


In [190]:
# Attributing average price and main category for the product '0_2245':
# df1['price']
values = {'price': 12.99, 'categ': '0'}

df1 = df1.fillna(value = values)

## 1.4. Check duplicates

In [191]:
print('Duplicated lines: {}'.format(df1.duplicated().sum()))

Duplicated lines: 126


In [157]:
df1[df1.duplicated() == True]

,id_prod,price,categ,date,client_id,sex,birth
34387,T_0,-1.0,0.0,test_2021-03-01 02:30:02.237443,ct_0,f,2001
54813,T_0,-1.0,0.0,test_2021-03-01 02:30:02.237412,ct_1,m,2001
57261,T_0,-1.0,0.0,test_2021-03-01 02:30:02.237439,ct_1,m,2001
58802,T_0,-1.0,0.0,test_2021-03-01 02:30:02.237429,ct_0,f,2001
60170,T_0,-1.0,0.0,test_2021-03-01 02:30:02.237446,ct_0,f,2001
62365,T_0,-1.0,0.0,test_2021-03-01 02:30:02.237412,ct_1,m,2001
71097,T_0,-1.0,0.0,test_2021-03-01 02:30:02.237446,ct_0,f,2001
72405,T_0,-1.0,0.0,test_2021-03-01 02:30:02.237424,ct_1,m,2001
75176,T_0,-1.0,0.0,test_2021-03-01 02:30:02.237443,ct_1,m,2001
76735,T_0,-1.0,0.0,test_2021-03-01 02:30:02.237442,ct_0,f,2001


### 1.4.1. Dropping duplicates

In [192]:
df1 = df1.drop_duplicates()

## 1.5. Odd values

In [193]:
# there are some test inputs that do not give any relevant information to the study case:
df1.sort_values(by = 'price').head()

,id_prod,price,categ,date,client_id,sex,birth
2365,T_0,-1.0,0,test_2021-03-01 02:30:02.237446,ct_1,m,2001
109598,T_0,-1.0,0,test_2021-03-01 02:30:02.237437,ct_1,m,2001
137506,T_0,-1.0,0,test_2021-03-01 02:30:02.237432,ct_0,f,2001
293107,T_0,-1.0,0,test_2021-03-01 02:30:02.237421,ct_0,f,2001
59043,T_0,-1.0,0,test_2021-03-01 02:30:02.237413,ct_0,f,2001


In [194]:
df1 = df1[df1['price'] > 0] # keep only product values above $0

In [195]:
print('Quantity of lines after dropping duplicates and odd values: {}'.format(df1.shape[0]))
print('Quantity of lines removed: {}'.format(337016 - df1.shape[0]))

Quantity of lines after dropping duplicates and odd values: 336816
Quantity of lines removed: 200


## 1.6. Data Types

In [196]:
df1.dtypes

id_prod       object
price        float64
categ         object
date          object
client_id     object
sex           object
birth          int64
dtype: object

### 1.6.1. Changing Data Types

In [197]:
df1['categ'] = np.where(df1['categ'].isnull(), df1['categ'], df1['categ'].astype(str).str[0])

In [202]:
df1['date'] = pd.to_datetime(df1['date'], format = '%Y-%m-%d')

## 1.7. Feature Engineering

In [203]:
df1.head()

,id_prod,price,categ,date,client_id,sex,birth,age,year,month,day,hour,period_of_day
0,0_1483,4.99,0,2021-04-10 18:37:28.723910,c_4450,f,1977,43,2021,4,10,18,night
1,2_226,65.75,2,2022-02-03 01:55:53.276402,c_277,f,2000,20,2022,2,3,1,morning_dawn
2,1_374,10.71,1,2021-09-23 15:13:46.938559,c_4270,f,1979,41,2021,9,23,15,afternoon
3,0_2186,4.20,0,2021-10-17 03:27:18.783634,c_4597,m,1963,57,2021,10,17,3,morning_dawn
4,0_1351,8.99,0,2021-07-17 20:34:25.800563,c_1242,f,1980,40,2021,7,17,20,night


In [200]:
# Create variables:
## Age
df1['age'] = df1['birth'].apply(lambda x: 2020 - x)

## Year of purchase
df1['year'] = df1['date'].dt.year

## Month of purchase
df1['month'] = df1['date'].dt.month

## Day of purchase
df1['day'] = df1['date'].dt.day

## Hour of purchase
df1['hour'] = df1['date'].dt.hour

## Time of day (morning / afternoon / evening)
df1['period_of_day'] = df1['hour'].apply(lambda x: 'morning_dawn' if x < 6
                                              else 'morning' if x < 12
                                              else 'afternoon' if x < 18
                                              else 'night')

## 1.8. Creating Dataset for another script

In [80]:
df1.to_csv('data/dataframe_for_analysis.csv', index = False)